In [1]:
# from IPython.display import HTML
# import urllib3
# HTML(urllib3.url('/Users/mahsa/OneDrive/Desktop/ML-projects/dogs-cats/style.css').read())

In this Tutorial blog, we will learn the basics of Convolutional Neural Networks (CNNs) and how to use them in Keras for the binary classification of dogs and cats images provided by <a href="https://www.kaggle.com/c/dogs-vs-cats/data">kaggle</a>. 

Why Keras? Keras is an open source neural network library that can be run on top of machine learning libraries such as Tensorflow or Theano. With Keras, you can wrtie high level code to bulid neural networks in just a few lines and Keras engine would covert the code to tensorflow or Theano code. Keras makes it very fast and easy to make protorypes and ready solutions while learning Tensrflow helps you understand the underlying sturcutre. 

For this project We will be using Keras with the Tenslorflow backend os before installing Keras, you need to install Tensorflow in your environmen first.

<h3>Importing Liberaries</h3>

So let's first import the python liberaries. As usual we have pandas for data wrangling. We will be using opencv for some image preprations. 

In [2]:
import numpy as np
import cv2 as cv2
import os
from tqdm import tqdm

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools


C:\Users\mahsa\AppData\Local\conda\conda\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h3>Loading the data</h3>

The train and test stes contain 25,000 and 12500 images of dogs and cats. So it would take some time to copy them on your machine.

In [3]:
train_path = '/Users/mahsa/OneDrive/Desktop/ML-projects/dogs-cats/train/'
test_path =  '/Users/mahsa/OneDrive/Desktop/ML-projects/dogs-cats/test/'

<h3>Pre-processing the images</h3>

Before We move to making a training model with Keras, we need to prepare the images as the input data. The images in both train adn test sets are in different sizes and we need to specify the single input shape in Keras sequnetial model we are going to use in this project. So all the input images should have the same shape. One of the tools to do image preparation is Opencv and can be used in different languages such aS C++ and python. Also, the Keras image agumentation instance that we will going use gets inputs as numpy arrays (I will explain keras image generator later in the process).
The function below uplaods and resizes each image to 200X200 and convert it to numpy array at the same time, then it stores the image array in a list (image_data). The function makes the labels list by going the list of image paths.

In [4]:
img_width = 128
img_height = 128

def label_img(img): 
    if 'dog.' in img: 
        return [0,1]                             
    elif 'cat.' in img:
        return [1,0]
        
def prepare_data(train_data_path):
    image_data = []
    for image_path in tqdm(train_data_path):
        label = label_img(image_path)
        img = cv2.resize(cv2.imread(image_path), (img_width,img_height), interpolation=cv2.INTER_CUBIC)
        image_data.append([np.array(img),np.array(label)])
    shuffle(image_data)
    return image_data

In [5]:
train_images = [train_path + i for i in os.listdir(train_path)] 
#test_images = [test_path + i for i in os.listdir(test_path)]

train_data = prepare_data(train_images[0:9000] + train_images[16000:25000])

X = np.array([j[0] for j in train_data])
Y = np.array([j[1] for j in train_data])

#test_data = prepare_data(test_images)

X_train, X_val, Y_train, Y_val = train_test_split(np.array(X), np.array(Y), test_size=0.2, random_state=121)

100%|██████████| 18000/18000 [04:00<00:00, 75.00it/s]


<h4>Image Augmentation</h4>

https://keras.io/preprocessing/image/

https://www.learnopencv.com/wp-content/uploads/2017/11/data-aug.png


One of the major challenges in making a model is finding enough data to train and avoid overfitting. Data Augmentation is a  technique to combat overfitting and help the model generalize better. Using Data Agumentation process, you can artificially create altered versions of the existing images to expand the training data set. The alternations include methods such as rotation, shearing, zooming, shifting etc. In keras the Agumentation process is done by using ImageDataGenerator instance.you can find the full list of augumentation parameters in ImageDataGenerator <a href="https://keras.io/preprocessing/image/">here</a>
The Augumentation parametters we are going ot use here are: 

Rotation, Horizontal filip, zooming ad shearing. another transforamtion that can be done by ImageDataGenerator is rescaling. The RGB pixel data is between 0-255 and with rescaling we reduce them to the range 0-1 in


In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=60,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
   )

val_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
   )

if you include featurewise_center or featurewise_std_normalization or zca_whitening in your ImageDataGenerator configuring, you would need to fit it on your data (both train and validation sets). This will calculate the internal data stats (std, mean and principal components) for the data those transformations specified in agumnetation process.

In [7]:
# train_datagen.fit(X_train)
# train_datagen.fit(X_val)

<h4>Generating data batches</h4>

The next step is generating bacthes of images data and their labels for our network which is done by The datagen.flow(). This function takes the numpy and labels array and make batches of agumented data.

In [8]:
batch_size = 64

train_generator = train_datagen.flow(X_train, Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(X_val, Y_val, batch_size=batch_size)

<h3>Making the Network</h3> 

<h4>Sequential vs. Functional models</h4>

There are two types models in Keras: functional and sequential. 

The sequential API allows you to design models layer by layer and connect each layer directly to the to the previous and next layers. You also need to specify only single input and outputs for the sequnetial models. 

On the othe hand, Functional API gives more felxibiltiy and allows to make deeper and more copmicated models and that are not squentional> Using functional API, you can connect layers to any other layer to make deeper models such as <a href="https://www.youtube.com/watch?v=K0uoBKBQ1gA">Residual Networks</a>. In addition, you can have multiple inputs and outputs for your network. 

Here we are going to use Sequnetial API to make a basic network.

The model is initialized by using Sequential() class. Now let's add layes to the model:

<h4>Convolutional Neural Network</h4>


For our image classifier model, we are going to a convolutional Neural Network as follow:

convoluted layer => Maxpooling => convoluted layer => maxpooling => fully connected layer => output layer

The model gets input as black and white 200X200 images, then convolutes and max pool the images in 2 sequnetial leyers. this is followed by a fully connceted layer and finally an output layer.

We add the first layer which is convolutional layer with the input shape of 200 height by 200 width and the channel number of 3 since the images are colorful. 

For both convolutional and fully connected layers, we use relu (rectified linear unit) as our activation function (why)
for this layer, I have chose the convolution window of 2X2 which will shift on the image data with the stride of 1X1 (). I will explain how convolutional and maxpooling work in details in another blog on using tensorflow classifer. You don't need to worry about the output size for the next layer which is a pooling layer and Keras will take care of calulation that.

Now that we convoluted the image, the next step is pooling. Pooling reduces the dimensions of output features which makes computation faster. we add a maxpooling layer that has a 2X2 pixel window and pickes the max number in image data by sliding over the images by 2 pixel in height and width.

Our model will consist of two Convolutional layers and two Maxppooling layers. After the first pooling layer, we double the number of input channels for the second convolutional layer(?). Afterwards, We flatten the output from the last maxpooling layer
for the fully connected layer we are going to add. Unlinke Tensorflow, we do not need to figure the size of output tesnsor to fellaten and keras takes care of that. 


Now we have out flattened input data to passs through a fully connected layer. This layer can be created using Dense class in Keras and we specify the number of nodes for this layer 100. 

for the first two convolutional layers we are using Relu as activation and for our fully connected layer we will activate nodes by a softmax function. 

We finally add the last layer which is the output layer which its size is the number of our binary classes of cat and dog.


To regularise our model, we add <a href="https://en.wikipedia.org/wiki/Dropout_(neural_networks)">Dropout</a> layer. This layer randomy selected neurons and drop 30% of them out of the training process in order to avoid overfitting in our model.

We are done! We were able to sucessfully create a CNN model architecture in just a few lines! you can see the model reperesnetion using summery instance:


In [9]:
n_classes =2

model = Sequential()

model.add(Conv2D(16, (3,3),
                 activation='relu',
                 input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 14400)             0         
__________

<h4>Compiling the model</h4>

Before we start traning our model, we need to sepcify learning process and the loss function. We configure these arguments by the 
compile method. We can also sepcify the evalution metric in the method. Here. we are going to use Adam optimizer, cross entropy as loss function and accuracy for our model evaluation. 

In [10]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

<h4>Training</h4>

Now it's time to train the model. This can be done by a single command in Keras. We are going to use fit_generator() method. If you are running your model on a GPU fit_generator is a better training method compared to fit() since it has higher efficiency. Fit_generator allows you to run parrel computations. So your CPU would be busy reading the data, doing the agumentation etc. while your GPU is being used to the model traning. If you are only using your CPU, you can use either fit or fit_generator.

Now we need to declare the training arugments for our fitting method. The first argument train_generator which includes the training samples and labels. The next argument is step_per_epoch which is the number of batches of data we are going to use in each epoch and epoch is the number of iterations we are going to train the model. We configure the validation generator and validation_steps the same way for the validation data set. The next argument to set is verbose which sepcifies how you would like to see the training set progressing in each epoch:

verbose = 0 : Shows nothing
verbose = 1: Shows a progress bar for each epoch
verbose = 2: Shows the number of epoch



The next argument is the batch size – we don’t have to explicitly handle the batching up of our data during training in Keras, rather we just specify the batch size and it does it for us (I have a post on mini-batch gradient descent if this is unfamiliar to you).  In this case we are using a batch size of 128.  Next we pass the number of training epochs (10 in this case).  The verbose flag, set to 1 here, specifies if you want detailed information being printed in the console about the progress of the training.  During training, if verbose is set to 1, the following is output to the console:

In [ ]:
model.fit_generator(train_generator, steps_per_epoch = len(X_train) // batch_size , 
                    validation_data=validation_generator, 
                    validation_steps= len(X_val) // batch_size, epochs=15, verbose=2 ,shuffle = True)

Epoch 1/15
 - 253s - loss: 0.6760 - acc: 0.5726 - val_loss: 0.6389 - val_acc: 0.6515
Epoch 2/15
 - 285s - loss: 0.6318 - acc: 0.6478 - val_loss: 0.6106 - val_acc: 0.6604
Epoch 3/15
 - 281s - loss: 0.6072 - acc: 0.6731 - val_loss: 0.5781 - val_acc: 0.7002
Epoch 4/15
